[Reference](https://medium.com/@etirismagazine/creating-an-ai-in-python-a-beginners-guide-9038a84e1fca)

# 1. Natural Language Processing (NLP):

In [1]:
import nltk

nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [2]:
sia = SentimentIntensityAnalyzer()

In [3]:
example_sentences = [
    "I love this movie, it's amazing!",
    "This movie is terrible, I hated it.",
    "The acting was great, but the plot was confusing.",
    "I didn't really like this movie, it was just okay.",
    "The special effects were impressive, but the story was weak."
]

for sentence in example_sentences:
    sentiment = sia.polarity_scores(sentence)
    print(sentence)
    print(sentiment)

I love this movie, it's amazing!
{'neg': 0.0, 'neu': 0.266, 'pos': 0.734, 'compound': 0.8516}
This movie is terrible, I hated it.
{'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.8074}
The acting was great, but the plot was confusing.
{'neg': 0.197, 'neu': 0.588, 'pos': 0.214, 'compound': 0.0516}
I didn't really like this movie, it was just okay.
{'neg': 0.207, 'neu': 0.624, 'pos': 0.169, 'compound': -0.1095}
The special effects were impressive, but the story was weak.
{'neg': 0.259, 'neu': 0.471, 'pos': 0.269, 'compound': -0.2144}


#     2. Recommendation Systems:

In [5]:
import pandas as pd

# Load the movie data
movies = pd.read_csv('movies.csv')

# Load the user ratings data
ratings = pd.read_csv('ratings.csv')

# Merge the movie and ratings data
movie_ratings = pd.merge(movies, ratings, on='movieId')

# Calculate the average rating for each movie
movie_ratings_grouped = movie_ratings.groupby('title')['rating'].mean().reset_index()

# Create a matrix of user ratings for each movie
movie_ratings_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

# Create a matrix of user ratings for each movie
movie_ratings_matrix = movie_ratings.pivot_table(index='userId', columns='title', values='rating')

# Find movies similar to a given movie
similar_movies = movie_ratings_matrix.corrwith(movie_ratings_matrix['Toy Story (1995)'])
similar_movies = similar_movies.dropna()
similar_movies_df = pd.DataFrame(similar_movies, columns=['correlation'])
similar_movies_df = similar_movies_df.sort_values('correlation', ascending=False)

# Suggest movies based on user preferences
user_ratings = movie_ratings_matrix.loc[1].dropna()
user_ratings_df = pd.DataFrame(user_ratings, columns=['rating'])
user_ratings_df = user_ratings_df.join(similar_movies_df)
suggested_movies = user_ratings_df.sort_values('correlation', ascending=False).head(10)

# 3. Image Recognition:

In [6]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Load the image data
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Preprocess the image data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

# Compile and train the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('Test accuracy:', test_acc)

#     4. Fraud Detection:

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Load the transaction data
data = pd.read_csv('transactions.csv')

# Preprocess the data
data = pd.get_dummies(data, columns=['location'])
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the machine learning model
model = RandomForestClassifier()

# Fit the model to the training data
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print('Accuracy:', accuracy)

# 5. Healthcare:

In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the image data
train_dir = 'train_images/'
train_df = pd.read_csv('train_labels.csv')

# Preprocess the image data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='id_code',
    y_col='diagnosis',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=train_dir,
    x_col='id_code',
    y_col='diagnosis',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Define the AI model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

# Compile and fit the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Evaluate the model
test_dir = 'test_images/'
test_df = pd.read_csv('test_labels.csv')

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=test_dir,
    x_col='id_code',
    y_col=None,
    target_size=(256, 256),
    batch_size=32,
    class_mode=None,
    shuffle=False
)

predictions = model.predict(test_generator)